In [1]:
import xarray as xr
import os
import numpy as np
import pandas as pd

In [15]:
# laptop
folder = "../../_data/p-dengue/weather_statistics-(reanalysis-era5-land)"

t2m_files = [f for f in os.listdir(os.path.join(folder,'t2m')) if f.endswith(".grib")]
d2m_files = [f for f in os.listdir(os.path.join(folder,'d2m')) if f.endswith(".grib")]
t2m_files.sort()
d2m_files.sort()

In [52]:
# saving
out_dir = os.path.join(folder, "rh")
os.makedirs(out_dir, exist_ok=True)

for i, year in enumerate(list(range(2015, 2025))):
    out_path = os.path.join(out_dir, f'rh_{year}.nc')
    if os.path.exists(out_path):
        print(year, "o")
        continue
    else:
        print(year, "x")

    # read in and remove empty last day of previous year
    t2m_path = os.path.join(os.path.join(folder,'t2m'), t2m_files[i])
    ds_t2m = xr.open_dataset(t2m_path, engine="cfgrib")
    ds_t2m = ds_t2m.isel(time=slice(1, None))

    d2m_path = os.path.join(os.path.join(folder,'d2m'), d2m_files[i])
    ds_d2m = xr.open_dataset(d2m_path, engine="cfgrib")
    ds_d2m = ds_d2m.isel(time=slice(1, None))

    # to dataframe and make sure indices align
    df_t2m  = ds_t2m["t2m"]
    df_d2m = ds_d2m["d2m"]
    df_t2m, df_d2m = xr.align(df_t2m, df_d2m, join="exact")

    # magnus formula for relative humidity
    df_t2m_c  = df_t2m  - 273.15
    df_d2m_c = df_d2m - 273.15
    df_rh = 100 * np.exp((17.625 * df_d2m_c) / (243.04 + df_d2m_c)) / np.exp((17.625 * df_t2m_c) / (243.04 + df_t2m_c))

    # to dataset
    ds_rh = df_rh.to_dataset(name="rh")
    ds_rh["rh"].attrs.update({"units": "%", "long_name": "Relative humidity (Magnus formula)"})

    # checking number of nan values (non land) are consistent
    n_nan_t2m = ds_t2m["t2m"].isnull().sum().item()
    n_nan_d2m = ds_d2m["d2m"].isnull().sum().item()
    n_nan_rh  = ds_rh["rh"].isnull().sum().item()

    print(n_nan_t2m==n_nan_d2m, n_nan_d2m==n_nan_rh, n_nan_rh==n_nan_t2m)
    
    # saving
    ds_rh.to_netcdf(out_path)

2015 o
2016 o
2017 o
2018 o
2019 o
2020 o
2021 o
2022 o
2023 o
2024 o
